Bunch of 3d stuffs

  **Table of Contents**

  <div id="toc"></div>
  <script type="text/javascript"
  src="https://raw.github.com/kmahelona/ipython_notebook_goodies/master/ipython_notebook_toc.js">
  </script>


# 3D Surface Subplots
https://plot.ly/python/3d-subplots/

In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

import numpy as np

x = np.linspace(-5, 80, 10)
y = np.linspace(-5, 60, 10)
xGrid, yGrid = np.meshgrid(y, x)
z = xGrid ** 3 + yGrid ** 3

scene = dict(
    xaxis=dict(
        gridcolor='rgb(255, 255, 255)',
        zerolinecolor='rgb(255, 255, 255)',
        showbackground=True,
        backgroundcolor='rgb(230, 230,230)'
    ),
    yaxis=dict(
        gridcolor='rgb(255, 255, 255)',
        zerolinecolor='rgb(255, 255, 255)',
        showbackground=True,
        backgroundcolor='rgb(230, 230,230)'
    ),
    zaxis=dict(
        gridcolor='rgb(255, 255, 255)',
        zerolinecolor='rgb(255, 255, 255)',
        showbackground=True,
        backgroundcolor='rgb(230, 230,230)'
    )
)

fig = tools.make_subplots(rows=2, cols=2,
                          specs=[[{'is_3d': True}, {'is_3d': True}],
                                 [{'is_3d': True}, {'is_3d': True}]])

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]
[ (2,1) scene3 ]  [ (2,2) scene4 ]



In [2]:
# adding surfaces to subplots.
fig.append_trace(dict(type='surface', x=x, y=y, z=z, colorscale='Viridis',
                      scene='scene1', showscale=False), 1, 1)
fig.append_trace(dict(type='surface', x=x, y=y, z=z, colorscale='RdBu',
                      scene='scene2', showscale=False), 1, 2)
fig.append_trace(dict(type='surface', x=x, y=y, z=z, colorscale='YIOrRd',
                      scene='scene3', showscale=False), 2, 1)
fig.append_trace(dict(type='surface', x=x, y=y, z=z, colorscale='YIGnBu',
                      scene='scene4', showscale=False), 2, 2)

# configure layout
fig['layout'].update(title='subplots with different colorscales',
                     height=800, width=800)
fig['layout']['scene1'].update(scene)
fig['layout']['scene2'].update(scene)
fig['layout']['scene3'].update(scene)
fig['layout']['scene4'].update(scene)


py.iplot(fig)

# 3d point clustering
https://plot.ly/python/3d-point-clustering/

In [3]:
import plotly.plotly as py
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/alpha_shape.csv')
df.head()

scatter = dict(
    mode = "markers",
    name = "y",
    type = "scatter3d",    
    x = df['x'], y = df['y'], z = df['z'],
    marker = dict( size=2, color="rgb(23, 190, 207)" )
)
clusters = dict(
    alphahull = 7,
    name = "y",
    opacity = 0.1,
    type = "mesh3d",    
    x = df['x'], y = df['y'], z = df['z']
)
layout = dict(
    title = '3d point clustering',
    scene = dict(
        xaxis = dict( zeroline=False ),
        yaxis = dict( zeroline=False ),
        zaxis = dict( zeroline=False ),
    )
)
fig = dict( data=[scatter, clusters], layout=layout )
# Use py.iplot() for IPython notebook
py.iplot(fig)

# 3d network graph
https://plot.ly/python/3d-network-graph/

The correct version of igraph is this
http://igraph.org/python/

In [4]:
%%bash
pip install python-igraph --user

In [5]:
import igraph as ig


In [6]:
import json
import urllib2

data = []
req = urllib2.Request("https://raw.githubusercontent.com/plotly/datasets/master/miserables.json")
opener = urllib2.build_opener()
f = opener.open(req)
data = json.loads(f.read())

print data.keys()

[u'nodes', u'links']


In [7]:
# number of nodes
N=len(data['nodes'])

# define list of edges and graph object from edges
L=len(data['links'])
Edges=[(data['links'][k]['source'], data['links'][k]['target']) for k in range(L)]

G=ig.Graph(Edges, directed=False)

print data['nodes'][0]

labels=[]
group=[]
for node in data['nodes']:
    labels.append(node['name'])
    group.append(node['group'])

{u'group': 1, u'name': u'Myriel'}


## Assign node posibion
Get the node positions, set by the Kamada-Kawai layout for 3D graphs:

In [8]:
layt=G.layout('kk', dim=3)

# layt is a list of three elements lists (the coordinates of nodes):
print layt[0]
print len(layt)

[7.217459422374638, 2.377425916869016, -1.942537513857466]
77


## Set data for the Plotly plot of the graph:

In [9]:
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]

## Define trace objects

In [10]:
import plotly.graph_objs as go

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=go.Line(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=go.Marker(symbol='dot',
                             size=6,
                             color=group,
                             colorscale='Viridis',
                             line=go.Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )


## Define layout object

In [11]:
axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of coappearances of characters in Victor Hugo's novel<br> Les Miserables (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=go.Scene(
         xaxis=go.XAxis(axis),
         yaxis=go.YAxis(axis),
         zaxis=go.ZAxis(axis),
        ),
     margin=go.Margin(
        t=100
    ),
    hovermode='closest',
    annotations=go.Annotations([
           go.Annotation(
           showarrow=False,
            text="Data source: <a href='http://bost.ocks.org/mike/miserables/miserables.json'>[1]</a>",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=go.Font(
            size=14
            )
            )
        ]),    )


## Plot!

In [12]:
data=go.Data([trace1, trace2])
fig=go.Figure(data=data, layout=layout)

py.iplot(fig)

# 3d filled line plots
https://plot.ly/python/3d-filled-line-plots/

In [13]:
import plotly.plotly as py
import pandas as pd

# The datasets' url. Thanks Jennifer Bryan!
url_csv = 'http://www.stat.ubc.ca/~jenny/notOcto/STAT545A/examples/gapminder/data/gapminderDataFiveYear.txt'

df = pd.read_csv(url_csv, sep='\t')
df.head()

countries = ['China', 'India', 'United States', 'Bangladesh', 'South Africa']
fill_colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854']
gf = df.groupby('country')

data = []

for country, fill_color in zip(countries[::-1], fill_colors):
    group = gf.get_group(country)
    years = group['year'].tolist()
    length = len(years)
    country_coords = [country] * length
    pop = group['pop'].tolist()
    zeros = [0] * length
    
    data.append(dict(
        type='scatter3d',
        mode='lines',
        x=years + years[::-1] + [years[0]],  # year loop: in incr. order then in decr. order then years[0]
        y=country_coords * 2 + [country_coords[0]],
        z=pop + zeros + [pop[0]],
        name='',
        surfaceaxis=1, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
        surfacecolor=fill_color,
        line=dict(
            color='black',
            width=4
        ),
    ))

layout = dict(
    title='Population from 1957 to 2007 [Gapminder]',
    showlegend=False,
    scene=dict(
        xaxis=dict(title=''),
        yaxis=dict(title=''),
        zaxis=dict(title=''),
        camera=dict(
            eye=dict(x=-1.7, y=-1.7, z=0.5)
        )
    )
)

fig = dict(data=data, layout=layout)

# IPython notebook
# py.iplot(fig, filename='filled-3d-lines')

py.iplot(fig)


# 3d scatter plots
https://plot.ly/python/3d-scatter-plots/
## Basic 3D Scatter Plot


In [14]:
import numpy as np
x, y, z = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 200).transpose()
trace1 = go.Scatter3d(
    x=x,y=y,z=z,
    mode='markers',
    marker=dict(size=12,
                opacity=0.8,
                line=dict(
                    color='rgba(217, 217, 217, 0.14)',
                    width=0.5),)
)

x2, y2, z2 = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 200).transpose()
trace2 = go.Scatter3d(
    x=x2,    y=y2,    z=z2,
    mode='markers',
    marker=dict(color='rgb(127, 127, 127)',
                size=12,
                opacity=0.9,
                symbol='circle',
                line=dict(
                    color='rgb(204, 204, 204)',
                    width=1),
    )
)

data = [trace1, trace2]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## 3D Scatter Plot with Colorscaling


In [15]:
x, y, z = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 400).transpose()

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=12,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# Projection of 3D surface
https://plot.ly/python/2d-projection-of-3d-surface/

## Set trace

In [16]:
xx=np.linspace(-3.5, 3.5, 100)
yy=np.linspace(-3.5, 3.5, 100)
x,y=np.meshgrid(xx, yy)
z=np.exp(-(x-1)**2-y**2)-10*(x**3+y**4-x/5)*np.exp(-(x**2+y**2))

colorscale=[[0.0, 'rgb(20,29,67)'],
           [0.1, 'rgb(28,76,96)'],
           [0.2, 'rgb(16,125,121)'],
           [0.3, 'rgb(92,166,133)'],
           [0.4, 'rgb(182,202,175)'],
           [0.5, 'rgb(253,245,243)'],
           [0.6, 'rgb(230,183,162)'],
           [0.7, 'rgb(211,118,105)'],
           [0.8, 'rgb(174,63,95)'],
           [0.9, 'rgb(116,25,93)'],
           [1.0, 'rgb(51,13,53)']]

# hover text for surface
textz=[['x: '+'{:0.5f}'.format(x[i][j])+'<br>y: '+'{:0.5f}'.format(y[i][j])+
        '<br>z: '+'{:0.5f}'.format(z[i][j]) for j in range(z.shape[1])] for i in range(z.shape[0])]

trace1= go.Surface(x=x, y=y, z=z,
                colorscale=colorscale,
                text=textz,
                hoverinfo='text',
                )


## set layout

In [17]:
axis = dict(showbackground=True, 
            backgroundcolor="rgb(230, 230,230)", 
            showgrid=False,    
            zeroline=False,  
            showline=False)

ztickvals=range(-6,4)
layout = go.Layout(title="Projections of a surface onto coordinate planes" , 
                autosize=False,
                width=700,
                height=600,
                scene=go.Scene(xaxis=go.XAxis(axis, range=[-3.5, 3.5]),
                               yaxis=go.YAxis(axis, range=[-3.5, 3.5]),
                               zaxis=go.ZAxis(axis , tickvals=ztickvals),
                               aspectratio=dict(x=1,y=1,z=0.95)
                           )
)


## Discretization of each Plane
The surface projections will be plotted in the planes of equations Z=np.min(z)-2, X=np.min(xx), respectively Y=np.min(yy).

In [18]:
z_offset=(np.min(z)-2)*np.ones(z.shape)#
x_offset=np.min(xx)*np.ones(z.shape)
y_offset=np.min(yy)*np.ones(z.shape)



## Define color for each xyz plane
Define the color functions and the color numpy arrays, C_z, C_x, C_y, corresponding to each plane:
Define the 3-tuples of coordinates to be displayed at hovering the mouse over the projections. The first two coordinates give the position in the projection plane, whereas the third one is used for assigning the color, just in the same way the coordinate z is used for the z-direction projection.

In [19]:
proj_z=lambda x, y, z: z#projection in the z-direction
colorsurfz=proj_z(x,y,z)
proj_x=lambda x, y, z: x
colorsurfx=proj_z(x,y,z)
proj_y=lambda x, y, z: y
colorsurfy=proj_z(x,y,z)

textx=[['y: '+'{:0.5f}'.format(y[i][j])+'<br>z: '+'{:0.5f}'.format(z[i][j])+
        '<br>x: '+'{:0.5f}'.format(x[i][j]) for j in range(z.shape[1])]  for i in range(z.shape[0])]
texty=[['x: '+'{:0.5f}'.format(x[i][j])+'<br>z: '+'{:0.5f}'.format(z[i][j]) +
        '<br>y: '+'{:0.5f}'.format(y[i][j]) for j in range(z.shape[1])] for i in range(z.shape[0])]  

tracex = go.Surface(z=z,
                x=x_offset,
                y=y,
                colorscale=colorscale,
                showlegend=False,
                showscale=False,
                surfacecolor=colorsurfx,
                text=textx,
                hoverinfo='text'
               )
tracey = go.Surface(z=z,
                x=x,
                y=y_offset,
                colorscale=colorscale,
                showlegend=False,
                showscale=False,
                surfacecolor=colorsurfy,
                text=texty,
                hoverinfo='text'
               )
tracez = go.Surface(z=z_offset,
                x=x,
                y=y,
                colorscale=colorscale,
                showlegend=False,
                showscale=False,
                surfacecolor=colorsurfx,
                text=textz,
                hoverinfo='text'
               )

## finally, plot

In [20]:
data=go.Data([trace1, tracex, tracey, tracez])
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)
